<a href="https://colab.research.google.com/github/michaelmoschitto/CoverLetterGPT/blob/main/src/models/CompleteSentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=8267a2466093e66378208185a6f1fb241b1eabefefe30ca616358f349b4b8f73
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

--2023-06-08 15:33:34--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.3MB/s    in 7.1s    

2023-06-08 15:33:41 (11.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [2]:
from huggingface_hub import notebook_login

notebook_login()

### Create the Dataset

In [3]:
NUM_TO_CAT = {0 : "Complete a sentence", 1 : "Read cover letter and answer questions", 2 : "Generate a cover letter with constraints"}
CAT_TO_NUM = {"Complete a sentence" : 0, "Read cover letter and answer questions" : 1, "Generate a cover letter with constraints" : 2}

In [7]:
import pandas as pd
data = {
    'prompt': [
        "Complete the sentence: I love to create machine learning models",
        "Complete the sentence: I love to create machine learning models",
        "Complete this statement: The best thing about summer is...",
        "Finish the following sentence: Life is like a...",
        "Complete this sentence",
        "Write the end of this sentence please: My name is ",
        "Write the end of this sentence please: I believe I am a great fit for the",
        "Write the end of this sentence please: Beyond my technical skills, my strong communication,",
        "Write the end of this sentence please: My technical experience, combined with a growth mindset and passion...",
        "Finish the following sentence: My technical experience, combined with a growth mindset and passion...",
        "Complete this phrase: My technical experience, combined with a growth mindset and passion...",
        "Complete this sentence: My technical experience, combined with a growth mindset and passion...",
        
        "Read the cover letter below and answer the questions.",
        "Please review the attached cover letter and answer the following questions.",
        "Answer the questions based on the cover letter provided.",
        "Answer these questions after reading this cover letter",
        "Review the cover letter below and answer the questions: ",
        "Please answer the following questions after review my cover letter",
        "Please answer the following questions after review my cover letter",
        "Please read and answer these questions based on my cover letter",
        "Read my cover letter. Next, answer these questions",
        "Review this cover letter. After, answer these questions:",
        "Answer the following question based on my cover letter",

        "Generate a cover letter expressing your interest in the position.",
        "Compose a cover letter for the job application with the given specifications.",
        "Write a cover letter demonstrating your skills and qualifications for the position.",
        "Write me a cover letter demonstrating my skills. these include: ",
        "Generate a cover letter and include these skills: ",
        "Write me a cover letter demonstrating the following skills for this job: ",
        "Compose me a cover letter that answers this call for employment: ",
        "Generate a cover letter responding to this job application: ",
        "Write a cover letter for the job posting and these qualifications: ",
        "Generate a cover letter responding to the following job application. Be sure to include these skills. ",
        "Write a cover letter for me. Include the following skills. Respond to the follow job posting: ",
    ],
    'target': [
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",

        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",


        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",

    ]
}

print(len(data["prompt"]), len(data["target"]))
train_data = pd.DataFrame(data)
train_data["target"] = train_data["target"].apply(lambda x: CAT_TO_NUM[x])
train_data.head(5)

34 34


,prompt,target
0,Complete the sentence: I love to create machin...,0
1,Complete the sentence: I love to create machin...,0
2,Complete this statement: The best thing about ...,0
3,Finish the following sentence: Life is like a...,0
4,Complete this sentence,0


In [8]:

import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(112)
df_train, df_test = train_test_split(train_data, test_size=0.2, random_state=42)

display(df_train.head())
display(df_test.head())

,prompt,target
12,Read the cover letter below and answer the que...,1
32,Generate a cover letter responding to the foll...,2
9,Finish the following sentence: My technical ex...,0
0,Complete the sentence: I love to create machin...,0
4,Complete this sentence,0


,prompt,target
15,Answer these questions after reading this cove...,1
19,Please read and answer these questions based o...,1
27,Generate a cover letter and include these skil...,2
26,Write me a cover letter demonstrating my skill...,2
8,Write the end of this sentence please: My tech...,0


### Create the model to fine tune

In [9]:

from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')

        for param in self.bert.encoder.layer[:2].parameters():
          param.requires_grad = False
        

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        # self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        # final_layer = self.relu(linear_output)

        return linear_output

### Create the dataset including encodings

In [31]:
from transformers import BertTokenizer, BertModel
import numpy as np
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = list(df["target"].values)
        self.texts = [text for text in df['prompt']]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'targets': label
        }
    
    def get_batch_labels(self, idx):
      label = self.labels[idx]
      encoded_label = tokenizer.encode_plus(
          label,
          add_special_tokens=True,
          padding='max_length',
          max_length=512,
          truncation=True,
          return_tensors="pt"
      )
      return encoded_label['input_ids'].squeeze(0)


### Train the model

In [36]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoModel, AutoTokenizer

# Step 1: Dataset Preparation
data = {
    'prompt': [
        "Complete the sentence: I love to create machine learning models",
        "Complete the sentence: I love to create machine learning models",
        "Complete this statement: The best thing about summer is...",
        "Finish the following sentence: Life is like a...",
        "Complete this sentence",
        "Write the end of this sentence please: My name is ",
        "Write the end of this sentence please: I believe I am a great fit for the",
        "Write the end of this sentence please: Beyond my technical skills, my strong communication,",
        "Write the end of this sentence please: My technical experience, combined with a growth mindset and passion...",
        "Finish the following sentence: My technical experience, combined with a growth mindset and passion...",
        "Complete this phrase: My technical experience, combined with a growth mindset and passion...",
        "Complete this sentence: My technical experience, combined with a growth mindset and passion...",
        
        "Read the cover letter below and answer the questions.",
        "Please review the attached cover letter and answer the following questions.",
        "Answer the questions based on the cover letter provided.",
        "Answer these questions after reading this cover letter",
        "Review the cover letter below and answer the questions: ",
        "Please answer the following questions after review my cover letter",
        "Please answer the following questions after review my cover letter",
        "Please read and answer these questions based on my cover letter",
        "Read my cover letter. Next, answer these questions",
        "Review this cover letter. After, answer these questions:",
        "Answer the following question based on my cover letter",

        "Generate a cover letter expressing your interest in the position.",
        "Compose a cover letter for the job application with the given specifications.",
        "Write a cover letter demonstrating your skills and qualifications for the position.",
        "Write me a cover letter demonstrating my skills. these include: ",
        "Generate a cover letter and include these skills: ",
        "Write me a cover letter demonstrating the following skills for this job: ",
        "Compose me a cover letter that answers this call for employment: ",
        "Generate a cover letter responding to this job application: ",
        "Write a cover letter for the job posting and these qualifications: ",
        "Generate a cover letter responding to the following job application. Be sure to include these skills. ",
        "Write a cover letter for me. Include the following skills. Respond to the follow job posting: ",
    ],
    'target': [
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",
        "Complete a sentence",

        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",
        "Read cover letter and answer questions",


        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",
        "Generate a cover letter with constraints",

    ]
}
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoModel, AutoTokenizer

train_data = pd.DataFrame(data)

# Step 2: Model Architecture
class SentenceCompletionModel(nn.Module):
    def __init__(self, base_model, hidden_size, dropout=0.1):
        super(SentenceCompletionModel, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, base_model.config.vocab_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)
        logits = self.fc(dropout_output)
        return logits

# Step 3: Tokenization
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 4: Data Loading
class SentenceCompletionDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        prompt = self.data['prompt'][index]
        target = self.data['target'][index]
        encoded_inputs = self.tokenizer(
            prompt,
            target,
            padding='max_length',
            max_length=512,
            truncation=True,
            return_tensors='pt'
train_data = pd.DataFrame(data)train_data = pd.DataFrame(data)

# Step 2: Model Architecture
class SentenceCompletionModel(nn.Module):
    def __init__(self, base_model, hidden_size, dropout=0.1):
        super(SentenceCompletionModel, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, base_model.config.vocab_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)
        logits = self.fc(dropout_output)
        return logits

# Step 3: Tokenization
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 4: Data Loading
class SentenceCompletionDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        prompt = self.data['prompt'][index]
        target = self.data['target'][index]
        encoded_inputs = self.tokenizer(
            prompt,
            target,
            padding='max_length',
            max_length=512,
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded_inputs['input_ids'].squeeze(),
            'attention_mask': encoded_inputs['attention_mask'].squeeze(),
            'target_ids': encoded_inputs['input_ids'].squeeze()  # Use input_ids as target for sentence completion
        }

train_dataset = SentenceCompletionDataset(train_data, tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)

# Step 5: Fine-tuning Setup
base_model = AutoModel.from_pretrained(model_name)
model = SentenceCompletionModel(base_model, hidden_size=base_model.config.hidden_size)
optimizer = Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 1

# Step 6: Training Loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        
        # Reshape target_ids to match the size of logits
        target_ids = target_ids[:, :logits.shape[1]]  
        
        loss = criterion(logits.view(-1, logits.shape[-1]),

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│             return_tensors='pt'                                                                  │
│                            ▲                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax. Perhaps you forgot a comma?

____

In [32]:
from huggingface_hub.repocard import DatasetCard
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoModel, AutoTokenizer

# Step 1: Dataset Preparation

# Step 2: Model Architecture
class SentenceCompletionModel(nn.Module):
    def __init__(self, base_model, hidden_size, dropout=0.1):
        super(SentenceCompletionModel, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
          outputs = self.base_model(input_ids, attention_mask=attention_mask)
          pooled_output = outputs.pooler_output
          dropout_output = self.dropout(pooled_output)
          logits = self.fc(dropout_output)
          return logits.squeeze(2)

# Step 3: Tokenization

# Step 4: Data Loading

# Step 5: Fine-tuning Setup
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
num_epochs = 1

model_name = 'sentence-transformers/bert-base-nli-mean-tokens'  # Example sentence transformer model
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)
model = SentenceCompletionModel(base_model, hidden_size=base_model.config.hidden_size).to(device)
optimizer = Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

train, val = Dataset(df_train), Dataset(df_test)
train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)


# # # Step 6: Training Loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device).float() 

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

# # Step 7: Evaluation

# # Step 8: Save and Load the Model
# torch.save(model.state_dict(), 'fine_tuned_sentence_transformer.pt')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 46>:54                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│ in forward:22                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

### Try out the trained model

In [ ]:
NUM_TO_CAT

{0: 'Complete a sentence',
 1: 'Read cover letter and answer questions',
 2: 'Generate a cover letter with constraints'}

In [ ]:


from transformers import BertTokenizer
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Load the trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Example texts for testing
examples = [
    "Please complete the following sentence:",
    "Please generate me a cover letter",
    "Complete this sentence: My technical experience, combined with a growth mindset and passion..."
]

# Tokenize and encode the examples
encoded_inputs = tokenizer.batch_encode_plus(examples, padding=True, truncation=True, return_tensors="pt")

# Extract the necessary inputs for your model
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]

# Move inputs to the appropriate device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Make predictions
with torch.no_grad():
    model.eval()
    outputs = model(input_ids, attention_mask)

# Get the predicted labels
predicted_labels = torch.argmax(outputs, dim=1).tolist()

# Map the predicted labels to their corresponding text
predicted_texts = [NUM_TO_CAT[label] for label in predicted_labels]

# Print the predicted labels for the examples
for text, label in zip(examples, predicted_texts):
    print(f"Text: {text}\nPredicted Label: {label}\n")


Text: Please complete the following sentence:
Predicted Label: Complete a sentence

Text: Please generate me a cover letter
Predicted Label: Read cover letter and answer questions

Text: Complete this sentence: My technical experience, combined with a growth mindset and passion...
Predicted Label: Complete a sentence

